In [46]:
import pandas as pd
import requests
import json
import dml
import prov.model
import datetime
import uuid
import csv 
from io import StringIO
import json
import pymongo
import numpy as np

In [47]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = [contributor + '.colleges_and_universities', contributor + '.landmarks']
writes = [contributor + '.cau_landmark_merge']

In [48]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [49]:
# get CAU_list from mongodb
CAU_list = list(repo[reads[0]].find())
CAU_df = pd.DataFrame(CAU_list)
CAU_list = np.array(CAU_df).tolist()

LandMark_list = list(repo[reads[1]].find())
LandMark_df = pd.DataFrame(LandMark_list)
LandMark_list = np.array(LandMark_df).tolist()

print(CAU_list[0])
print(LandMark_list[0])

['Fruit St, Boston, Massachusetts 02114', 'West End', '42.36259065', '-71.07014072', 'Massachusetts General Hospital Dietetic Internship', '365190', ObjectId('5c819e84647d4b10977648ed')]
['210 East Cottage Street', 'Exterior & Interior (parcel 2942)', 'James Blake House', 'Dorchester', '20', 51.0, ObjectId('5c819e85647d4b109776492a')]


In [50]:
# define relational models
def union(R, S):
    return R + S

def difference(R, S):
    return [t for t in R if t not in S]

def intersect(R, S):
    return [t for t in R if t in S]

def project(R, p):
    return [p(t) for t in R]

def select(R, s):
    return [t for t in R if s(t)]
 
def product(R, S):
    return [(t,u) for t in R for u in S]

def aggregate(R, f):
    keys = {r[0] for r in R}
    return [(key, f([v for (k,v) in R if k == key])) for key in keys]

In [51]:
# select and project the aimed list
cau_list = [(City, Name, Address) for [Address, City, latitude, longitude, Name, Zipcode, id] in CAU_list]
landmark_list = [(City, Name, Address) for [Address, Areas_Desi, Name, City, Petiton, Width, id] in LandMark_list]
cau_landmark_merge_list = cau_list + landmark_list
# print(cau_landmark_merge_list)
# cau_landmark_merge_list = [(City, Name, Address) for [Address, City, latitude, longitude, Name, Zipcode, id] in CAU_list 
#                           for [Address, Areas_Desi, Name, City, Petiton, Width, id] in LandMark_list]

columnName = ['Neighborhood', 'Name', 'Address']
df = pd.DataFrame(columns = columnName, data = cau_landmark_merge_list)
data = json.loads(df.to_json(orient="records"))

In [52]:
repo.dropCollection('cau_landmark_merge')
repo.createCollection('cau_landmark_merge')
repo[writes[0]].insert_many(data)

In [53]:
repo[writes[0]].metadata({'complete': True})
print(repo[writes[0]].metadata())
[record for record in repo[writes[0]].find()]

{'_id': ObjectId('5c81a77c647d4b0e60b3b2a4'), 'complete': True}


[{'_id': ObjectId('5c81a77c647d4b0e60b3b1cd'),
  'Neighborhood': 'West End',
  'Name': 'Massachusetts General Hospital Dietetic Internship',
  'Address': 'Fruit St, Boston, Massachusetts 02114'},
 {'_id': ObjectId('5c81a77c647d4b0e60b3b1ce'),
  'Neighborhood': 'Boston',
  'Name': 'Suffolk University',
  'Address': '8 Ashburton Pl Beacon Hill, Boston, Massachusetts 02108'},
 {'_id': ObjectId('5c81a77c647d4b0e60b3b1cf'),
  'Neighborhood': 'South End',
  'Name': 'Benjamin Franklin Institute of Technology',
  'Address': '41 Berkeley St, Boston, Massachusetts 02116'},
 {'_id': ObjectId('5c81a77c647d4b0e60b3b1d0'),
  'Neighborhood': 'Charlestown',
  'Name': 'Bunker Hill Community College',
  'Address': '250 New Rutherford Ave, Boston, Massachusetts 02129'},
 {'_id': ObjectId('5c81a77c647d4b0e60b3b1d1'),
  'Neighborhood': 'Charlestown',
  'Name': 'MGH Institute of Health Professions',
  'Address': '36 1st Avenue, Boston, Massachusetts 02129'},
 {'_id': ObjectId('5c81a77c647d4b0e60b3b1d2'),
  

In [54]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 3, 7, 18, 21, 27, 969706),
 'end': datetime.datetime(2019, 3, 7, 18, 21, 46, 988306)}

In [55]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#cau_landmark_merge', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
resource = doc.entity('bdp:wc8w-nujj', {'prov:label':'311, Service Requests', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
get_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(get_names, this_script)
doc.usage(get_names, resource, startTime, None,
    {prov.model.PROV_TYPE:'ont:Retrieval',
    'ont:Computation':'Data cleaning'
    }
)

fp = doc.entity('dat:'+contributor+'#cau_landmark_merge', {prov.model.PROV_LABEL:'CAU and Landmark Merge', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(fp, this_script)
doc.wasGeneratedBy(fp, get_names, endTime)
doc.wasDerivedFrom(fp, resource, get_names, get_names, get_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [56]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#cau_landmark_merge, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:wc8w-nujj, [prov:label="311, Service Requests", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuid229d1da3-1971-45cf-9287-bf0be7ffe9b1, -, -)
  wasAssociatedWith(log:uuid229d1da3-1971-45cf-9287-bf0be7ffe9b1, alg:mmao95_Dongyihe_weijiang_zhukk#cau_landmark_merge, -)
  used(log:uuid229d1da3-1971-45cf-9287-bf0be7ffe9b1, bdp:wc8w-nujj, -, [prov:type="ont:Retrieval", ont:Computation="Data cleaning"])
  entity(dat:mmao95_Dongyihe_weijiang_zhukk#cau_landmark_merge, [prov:label="CAU and Landmark Merge", prov:type="ont:DataSet"])
  wasAttributedTo(dat:mmao95_Dongyihe_weijiang_zhukk#cau_landmark_me